#**Article Recommender App**


This is a collab sheet for data merging and unzipping of the codes . Following have cpdes for Part 1 . After preparing this data it can be used for implementing a react native application .


Collab sheet's code by satvik .

####***Part - A (1)  Importing data from .kaggle.com***

##**Steps** :--

### **(1)** Working on terminal first 

Installing kaggle using *pip install* command .

######***Remember to install the kaggle.json file before working on below given codes*** . 👇👇👇

In [1]:
!pip install kaggle

making directory and giving permissions .

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

Now installing dataset zip file to work .

In [3]:
!kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop

 61% 5.00M/8.20M [00:00<00:00, 7.56MB/s]
100% 8.20M/8.20M [00:00<00:00, 23.6MB/s]


Listing files to check .

In [4]:
!ls

articles-sharing-reading-from-cit-deskdrop.zip	kaggle.json  sample_data


Unzipping the zipped file to get the .csv files for working .

In [5]:
!unzip articles-sharing-reading-from-cit-deskdrop.zip

Archive:  articles-sharing-reading-from-cit-deskdrop.zip
  inflating: shared_articles.csv     
  inflating: users_interactions.csv  


Listing

In [6]:
!ls

articles-sharing-reading-from-cit-deskdrop.zip	shared_articles.csv
kaggle.json					users_interactions.csv
sample_data


**(2)** Importing dependencies and reading the csv files .

In [7]:
import pandas as pd 
import numpy as np 

df1=pd.read_csv('shared_articles.csv')
df2=pd.read_csv('users_interactions.csv')

**(3)** Checking the headers of df1 variable's csv file .

In [8]:
df1.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


**(3)** Checking the headers of df2 variable's csv file .

In [9]:
df2.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


####***PART - B (2) Demographic filtering***

*Now , we are going to understand the data that we got in the last project (article’s data), analyse it and then perform demographic filtering on it!*


######**Demographic Filtering (DF) technique uses the demographic data of a user to determine which items may be appropriate for recommendation.**######

In [10]:
df1 = df1[df1['eventType'] == 'CONTENT SHARED']
df1.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en


In [11]:
df2.shape

(72312, 8)

In [12]:
df1.shape

(3047, 13)

In [13]:
def find_total_events(df1_row):
  total_likes = df2[(df2["contentId"] == df1_row["contentId"]) & (df2["eventType"] == "LIKE")].shape[0]
  total_views = df2[(df2["contentId"] == df1_row["contentId"]) & (df2["eventType"] == "VIEW")].shape[0]
  total_bookmarks = df2[(df2["contentId"] == df1_row["contentId"]) & (df2["eventType"] == "BOOKMARK")].shape[0]
  total_follows = df2[(df2["contentId"] == df1_row["contentId"]) & (df2["eventType"] == "FOLLOW")].shape[0]
  total_comments = df2[(df2["contentId"] == df1_row["contentId"]) & (df2["eventType"] == "COMMENT CREATED")].shape[0]
  return total_likes + total_views + total_bookmarks + total_follows + total_comments

df1["total_events"] = df1.apply(find_total_events, axis=1)

df1.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang,total_events
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en,1
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en,1
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en,13
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en,0
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en,2


In [14]:
df1 = df1.sort_values(['total_events'], ascending=[False])

df1.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang,total_events
3096,1487246811,CONTENT SHARED,-4029704725707465084,6013226412048763966,-6569695881431984742,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,HTML,http://www.cnbc.com/2016/12/21/former-google-c...,Former Google career coach shares a visual tri...,"If you want 2017 to be an exciting year, desig...",en,433
1671,1467813728,CONTENT SHARED,-133139342397538859,4918484843075254252,-5701227433817087697,NaN,NaN,NaN,HTML,http://gq.globo.com/Prazeres/Poder/Carreira/no...,"Novo workaholic trabalha, pratica esportes e t...",Novo workaholic não abre mão do esporte e da f...,pt,315
1814,1468867647,CONTENT SHARED,-6783772548752091658,4918484843075254252,-8995217520473210153,NaN,NaN,NaN,HTML,http://www.caroli.org/livro-retrospectivas-div...,Livro: Retrospectivas Divertidas,"Neste livro, nós fornecemos um conjunto de fer...",pt,294
1317,1465484901,CONTENT SHARED,8657408509986329668,-8020832670974472349,838596071610016700,NaN,NaN,NaN,HTML,https://medium.com/practical-blend/pull-reques...,Pull request first - Practical Blend,Pull request first After two years of working ...,en,294
588,1461629452,CONTENT SHARED,-6843047699859121724,7527226129639571966,-1297230017812472163,NaN,NaN,NaN,HTML,https://medium.com/@jeffersoncn/ganhe-6-meses-...,"Ganhe 6 meses de acesso ao Pluralsight, maior ...","Ganhe 6 meses de acesso ao Pluralsight, maior ...",pt,281


####**PART - C**

#####*Content based filtering*

WHAT IS CONTENT BASED FILTERING ?

**[ Ans ] :--** *Content-based filtering is a type of recommender system that attempts to guess what a user may like based on that user's activity. Content-based filtering makes recommendations by using keywords and attributes assigned to objects in a database (e.g., items in an online marketplace) and matching them to a user profile.*

*..src=www.upwork.com*

Codes by : satvik

In [29]:
#df1[['title', 'contentType', 'url', 'text', 'lang','contentId']].head(3) 			

In [28]:
def convert_lowercase(x):
  if isinstance(x, str):
      return x.lower()
  else:
      return ''

df1["title"] = df1["title"].apply(convert_lowercase)

df1.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang,total_events
3096,1487246811,CONTENT SHARED,-4029704725707465084,6013226412048763966,-6569695881431984742,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,HTML,http://www.cnbc.com/2016/12/21/former-google-c...,former google career coach shares a visual tri...,"If you want 2017 to be an exciting year, desig...",en,433
1671,1467813728,CONTENT SHARED,-133139342397538859,4918484843075254252,-5701227433817087697,NaN,NaN,NaN,HTML,http://gq.globo.com/Prazeres/Poder/Carreira/no...,"novo workaholic trabalha, pratica esportes e t...",Novo workaholic não abre mão do esporte e da f...,pt,315
1814,1468867647,CONTENT SHARED,-6783772548752091658,4918484843075254252,-8995217520473210153,NaN,NaN,NaN,HTML,http://www.caroli.org/livro-retrospectivas-div...,livro: retrospectivas divertidas,"Neste livro, nós fornecemos um conjunto de fer...",pt,294
1317,1465484901,CONTENT SHARED,8657408509986329668,-8020832670974472349,838596071610016700,NaN,NaN,NaN,HTML,https://medium.com/practical-blend/pull-reques...,pull request first - practical blend,Pull request first After two years of working ...,en,294
588,1461629452,CONTENT SHARED,-6843047699859121724,7527226129639571966,-1297230017812472163,NaN,NaN,NaN,HTML,https://medium.com/@jeffersoncn/ganhe-6-meses-...,"ganhe 6 meses de acesso ao pluralsight, maior ...","Ganhe 6 meses de acesso ao Pluralsight, maior ...",pt,281


In [30]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df1['title'])

In [31]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [32]:
df1 = df1.reset_index()
indices = pd.Series(df1.index, index=df1['contentId'])

In [33]:
def get_recommendations(contentId, cosine_sim):
    idx = indices[contentId]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df1['contentId'].iloc[movie_indices]

In [36]:
get_recommendations(-7292285110016212249, cosine_sim2)

286    -7986975759046428357
885     -724699218170106849
1989    7372398343609401828
1873    8084284001249507595
149     2727743992157210358
215     1436883058900979473
695    -6778286518036232293
959    -6195775145989617417
1254    5304344058087060692
2817   -7926018713416777892
Name: contentId, dtype: int64

In [37]:
from google.colab import files

df1.to_csv('data.csv')
files.download('data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>